In [2]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import unicodedata

In [3]:
fecha = pd.read_csv('datasets/peliculas_por_estreno.csv', parse_dates = ['release_date'])
popularidad = pd.read_csv('datasets/peliculas_por_popularidad.csv')
votos = pd.read_csv('datasets/peliculas_por_votos.csv')
actores = pd.read_csv('datasets/peliculas_por_actor.csv')
directores = pd.read_csv('datasets/peliculas_por_director.csv', parse_dates=['release_date'])


In [ ]:
df_movies = pd.read_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\df_movies_final.csv", parse_dates = ['release_date'])

In [ ]:
#df_credits = pd.read_csv(r"C:\Users\romin\Documents\PROGRAMACION\CIENCIA DE DATOS\HENRY\LABS\PROYECTO INDIVIDUAL\PI 01 MLOps\df_credits2_final.csv")

In [ ]:
#del df_credits['Unnamed: 0']

In [4]:
def trad_fechas(dato):
    diames_norm = unicodedata.normalize('NFD', dato)
    diames_norm = diames_norm.encode('ascii', 'ignore').decode('utf-8')
    diames_norm = diames_norm.lower()

    mapping = {
        'lunes': 'Monday',
        'martes': 'Tuesday',
        'miercoles': 'Wednesday',
        'jueves': 'Thursday',
        'viernes': 'Friday',
        'sabado': 'Saturday',
        'domingo': 'Sunday',
        'enero': 'January',
        'febrero': 'February',
        'marzo': 'March',
        'abril': 'April',
        'mayo': 'May',
        'junio': 'June',
        'julio': 'July',
        'agosto': 'August',
        'septiembre': 'September',
        'octubre': 'October',
        'noviembre': 'November',
        'diciembre': 'December',
    }

    return mapping.get(diames_norm, diames_norm)




CONSULTA 1
def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.



In [5]:
def cantidad_filmaciones_mes(mes:str):
    mes_num = datetime.strptime(trad_fechas(mes), "%B").month
    pel_mes = fecha[fecha['release_date'].dt.month == mes_num]
    cantidad = len(pel_mes)
    return ('{} películas fueron estrenadas en el mes de {}'.format(cantidad, mes))

In [6]:
cantidad_filmaciones_mes('enero')

'5912 películas fueron estrenadas en el mes de enero'



CONSULTA 2
def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.



In [7]:
def cantidad_filmaciones_dia(dia:str):
    dia_tr = trad_fechas(dia)
    pel_dia = fecha[fecha['release_date'].dt.strftime('%A') == dia_tr] 
    cantidaddia = len(pel_dia)
    return ('{} películas fueron estrenadas en los días {}'.format(cantidaddia, dia))

In [8]:
cantidad_filmaciones_dia('miercoles')

'7035 películas fueron estrenadas en los días miercoles'



CONSULTA 3
def score_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.




In [11]:
def score_titulo(pelicula: str):
    filtro = popularidad['title'] == pelicula
    score = popularidad.loc[filtro, 'popularity'].values[0] if filtro.any() else None
    año = popularidad.loc[filtro, 'release_year'].values[0] if filtro.any() else None
    return ('La película {} fue estrenada en el año {} con un score de {}'.format(pelicula, año, round(score, 2)))

In [12]:
score_titulo('Avatar')

'La película Avatar fue estrenada en el año 2009 con un score de 185.07'




CONSULTA 4
def votos_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.




In [13]:
def votos_titulo(pelicula: str):
    filtro = (votos['title'] == pelicula) 
    df_filtrado = votos.loc[filtro].sort_values('vote_count', ascending=False)
    
    vote_count = df_filtrado.iloc[0]['vote_count']
    
    if vote_count < 2000:
        return 'La película {} no cuenta con puntajes suficientes.'.format(pelicula)
    
    total_votos = df_filtrado.iloc[0]['vote_count']
    promedio = df_filtrado.iloc[0]['vote_average']
    año = df_filtrado.iloc[0]['release_year']
    
    return ('La película {} fue estrenada en el año {}. La misma cuenta con un total de {} valoraciones, con un promedio de {}'.format(pelicula, año, total_votos, promedio))


In [14]:
votos_titulo('Wonder Woman')

'La película Wonder Woman fue estrenada en el año 2017. La misma cuenta con un total de 5025.0 valoraciones, con un promedio de 7.2'



CONSULTA 5
def get_actor( nombre_actor ): Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.




In [17]:
def get_actor(actor: str):
    cantidad = actores['cast'].apply(lambda x: actor in x).sum()
    retorno = actores.loc[actores['cast'].apply(lambda x: actor in x), 'return'].sum()
    promedio = actores.loc[actores['cast'].apply(lambda x: actor in x), 'return'].mean()
    return ('El actor {} ha participado de {} de filmaciones, el mismo ha conseguido un retorno de {}, con un promedio de {} por filmación'. format(actor, cantidad, round(retorno,2), round(promedio,2)))


In [18]:
get_actor('Tom Hanks')

'El actor Tom Hanks ha participado de 71 de filmaciones, el mismo ha conseguido un retorno de 178.85, con un promedio de 2.52 por filmación'



CONSULTa 6
def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.




In [21]:
def get_director(nombre_director: str):
    df_filtrado = directores[directores['crew'].apply(lambda x: nombre_director in x)][['title', 'release_year', 'return', 'budget', 'revenue']]
    peliculas = df_filtrado['title']
    año = df_filtrado['release_year']
    retorno_pelicula = df_filtrado['return']
    budget_pelicula = df_filtrado['budget']
    revenue_pelicula = df_filtrado['revenue']
    retorno_total = df_filtrado['revenue'].sum()
    return {'director': nombre_director, 'retorno_total_director':retorno_total, 'peliculas':peliculas, 'anio': año, 'retorno_pelicula': retorno_pelicula, 'budget_pelicula': budget_pelicula, 'revenue_pelicula': revenue_pelicula}


In [22]:
get_director("Martin Scorsese")

{'director': 'Martin Scorsese',
 'retorno_total_director': 1990982842.0,
 'peliculas': 15                                                  Casino
 109                                            Taxi Driver
 407                                   The Age of Innocence
 1171                                            GoodFellas
 1186                                           Raging Bull
 1296                                             Cape Fear
 1648                                                Kundun
 1913                         The Last Temptation of Christ
 2361                                    The Color of Money
 2861                                 Bringing Out the Dead
 4500                                      New York Stories
 5143                                         Boxcar Bertha
 5253                                        The Last Waltz
 5822                                     Gangs of New York
 5867                                    The King of Comedy
 7270         